In [30]:
import torch
import clip
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import tensorflow as tf
import tensorflow_hub as hub


In [31]:
#Faster R-CNN
faster_rcnn_url = "https://www.kaggle.com/models/google/faster-rcnn-inception-resnet-v2/tensorFlow1/faster-rcnn-openimages-v4-inception-resnet-v2/1"
model_frcnn = hub.load(faster_rcnn_url).signatures['default']

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [32]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_clip, preprocess = clip.load("ViT-B/32", device=device)

In [59]:
def preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    image = image.resize((640, 640))  # Resize to 640x640
    image_np = np.array(image)
    image_np = tf.convert_to_tensor(image_np, dtype=tf.float32)
    image_np = tf.image.per_image_standardization(image_np)  # Standardize to zero mean and unit variance
    image_np = tf.expand_dims(image_np, 0)  # Add batch dimension
    return image_np

def preprocess_image_clip(image):
    return preprocess(image).unsqueeze(0).to(device)

def object_detection(image_tensor):
    results = model_frcnn(image_tensor)
    
    boxes = results['detection_boxes'].numpy()
    scores = results['detection_scores'].numpy()
    classes = results['detection_class_entities'].numpy()
    
    print("Detection boxes shape:", boxes.shape)
    print("Detection scores shape:", scores.shape)
    print("Detection classes shape:", classes.shape)
    
    threshold = 0.3
    valid_indices = np.where(scores > threshold)[0]
    
    boxes = boxes[valid_indices]
    scores = scores[valid_indices]
    classes = classes[valid_indices]
    
    return boxes, scores, classes

In [72]:
texts = [ 'Con chim',   
         'camera',
         'người',
         'mắt kính',
         'chó',
         'mèo',
         'smartphone']

text_inputs = clip.tokenize(texts).to(device)

In [73]:
def classify_objects(image,boxes):
    results = []
    for box in boxes:
        y_min, x_min, y_max, x_max = box
        x_min *= image.width
        x_max *= image.width
        y_min *= image.height
        y_max *= image.height
        
        x_min = max(0, x_min)
        x_max = min(image.width, x_max)
        y_min = max(0, y_min)
        y_max = min(image.height, y_max)
        
        object_image = image.crop((x_min, y_min, x_max, y_max))
        object_image_input = preprocess_image_clip(object_image)
        
        with torch.no_grad():
            image_features = model_clip.encode_image(object_image_input)
            text_features = model_clip.encode_text(text_inputs)
            
            image_features /= image_features.norm(dim=-1, keepdim=True)
            text_features /= text_features.norm(dim=-1, keepdim=True)
            
            similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
        
        print("Similarity scores:", similarity)
        
        values, indices = similarity.topk(1)
        class_name = texts[indices.item()]
        class_scores = similarity[0].cpu().numpy()
        results.append((box, class_name, class_scores))
    return results

In [74]:
def draw(image_path, boxes, labels):
    image = Image.open(image_path)
    fig, ax = plt.subplots(1, figsize=(12, 9))
    ax.imshow(image)
    
    for box, label in zip(boxes, labels):
        y_min, x_min, y_max, x_max = box
        x_min *= image.width
        x_max *= image.width
        y_min *= image.height
        y_max *= image.height
    
        rect = patches.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min, linewidth=1, edgecolor='r', facecolor='none')
        ax.add_patch(rect)
        ax.text(x_min, y_min, label, color='r', fontsize=12, bbox=dict(facecolor='white', alpha=0.5))
        
    plt.axis('off')
    plt.show()

image_path = '../webapp/static/upload/au.jpg'
image_np = preprocess_image(image_path)
image = Image.open(image_path)

boxes, scores, classes = object_detection(image_np)
if len(boxes) == 0:
    print("No objects detected.")
else:
    labels = classify_objects(image, boxes)
    for box, label, class_scores in labels:
        print(f"Detected object: {label}")
        for text, score in zip(texts, class_scores):
            print(f" - Text: {text}, Score: {score:.2f}")
    draw(image_path, [box for box, _ in labels], [label for _, label in labels])

Detection boxes shape: (100, 4)
Detection scores shape: (100,)
Detection classes shape: (100,)
Similarity scores: tensor([[0.0014, 0.0961, 0.0354, 0.3118, 0.3236, 0.2214, 0.0104]])
Similarity scores: tensor([[0.0012, 0.1413, 0.0143, 0.1771, 0.2719, 0.3804, 0.0137]])
Similarity scores: tensor([[0.0035, 0.4186, 0.0073, 0.0543, 0.2705, 0.2369, 0.0089]])
Similarity scores: tensor([[0.0009, 0.0593, 0.0671, 0.4698, 0.2700, 0.1244, 0.0085]])
Similarity scores: tensor([[0.0030, 0.1907, 0.0438, 0.1676, 0.3816, 0.2020, 0.0115]])
Similarity scores: tensor([[0.0165, 0.3531, 0.0081, 0.0264, 0.3670, 0.1091, 0.1197]])


ValueError: too many values to unpack (expected 2)